In [4]:
import pandas as pd
import os
import numpy as np 
import tensorflow as tf 
import cv2
from os import walk
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Better copy data from Drive as an archive
zip_path = '/content/gdrive/My Drive/emotion_classification/data_emotion.zip'

!cp "{zip_path}" .

!unzip -q data_emotion.zip

# Remove .zip file after you unzip it
!rm data_emotion.zip

# Make sure it's there
!ls

replace data/test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
data  gdrive  sample_data


In [6]:
IMG_SIZE=(48,48)

In [7]:
## Train 
listeRepertories = []
for (repertoire, sousRepertoires, fichiers) in walk("data/train"):
    listeRepertories.extend((sousRepertoires))

classDict={}
for i in range(len(listeRepertories)):
    classDict[listeRepertories[i]]=i

data={'path':[],'classe':[]}
for rep in listeRepertories:

    for (repertoire, sousRepertoires, fichiers) in walk(f"data/train/{rep}"):
        i=0
        for fichier in fichiers:
          if i<40000:
            i=i+1
            data['path'].append(f"data/train/{rep}/{fichier}")
            data['classe'].append(classDict[rep])

dataTrain=pd.DataFrame(data)

## Test

listeRepertories = []
for (repertoire, sousRepertoires, fichiers) in walk("data/test"):
    listeRepertories.extend((sousRepertoires))

classDict={}
for i in range(len(listeRepertories)):
    classDict[listeRepertories[i]]=i

data={'path':[],'classe':[]}
for rep in listeRepertories:
      for (repertoire, sousRepertoires, fichiers) in walk(f"data/test/{rep}"):
        i=0 
        for fichier in fichiers:
          if i<40000:
            i=i+1
            data['path'].append(f"data/test/{rep}/{fichier}")
            data['classe'].append(classDict[rep])

dataTest=pd.DataFrame(data)



print("Nos classes sont les suivantes :")
print(classDict)

Nos classes sont les suivantes :
{'sad': 0, 'neutral': 1, 'surprise': 2, 'angry': 3, 'disgust': 4, 'happy': 5, 'fear': 6}


On a regroupé tout les chemins de nos images ainsi que leurs classes dans le dataframe suivant :

In [8]:
dataTrain.head()

,path,classe
0,data/train/sad/Training_47792642.jpg,0
1,data/train/sad/Training_8925619.jpg,0
2,data/train/sad/Training_49922991.jpg,0
3,data/train/sad/Training_23882155.jpg,0
4,data/train/sad/Training_68307224.jpg,0


On va maintenant charger nos images afin de pouvoir les utiliser pour notre entrainement.

In [9]:
## Train

X_train=[]

for imgPath,imgClass in dataTrain.values:
    #img=load_img(imgPath,target_size=IMG_SIZE,color_mode='grayscale')
    #x=img_to_array(img)
    x=cv2.imread(imgPath)

    X_train.append(x)


## Test

X_test=[]

for imgPath,imgClass in dataTest.values:
    #img=load_img(imgPath,target_size=IMG_SIZE,color_mode='grayscale')
    #x=img_to_array(img)
    x=cv2.imread(imgPath)

    X_test.append(x)



len(X_train)

28709

In [10]:
y_train=dataTrain['classe'].values
y_test=dataTest['classe'].values

dataTrain=[]
dataTest=[]

In [11]:
X_train=np.array(X_train)
X_test=np.array(X_test)

X_train=(X_train.astype('float')/255)
X_test=(X_test.astype('float')/255)

In [12]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
model=tf.keras.Sequential()

model.add(Conv2D(32,(7,7),padding='same',kernel_initializer='he_normal',
                 input_shape=(48,48,3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(7,7),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(7,7),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(256,(7,7),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(512,kernel_initializer='he_normal'))
model.add(Activation('elu'))


model.add(Dense(256,kernel_initializer='he_normal'))
model.add(Activation('elu'))

#Block-7
model.add(Dense(7,kernel_initializer='he_normal'))
model.add(Activation('softmax'))



In [22]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adamax(),
    metrics=['accuracy']
)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 48, 48, 32)        4736      
_________________________________________________________________
activation_14 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 64)        100416    
_________________________________________________________________
activation_15 (Activation)   (None, 24, 24, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 24, 24, 64)       

In [23]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]


history = model.fit(X_train,y_train,
                    epochs=100,
                    shuffle=True,
                    validation_data=(X_test,y_test),
                    callbacks=callbacks)

Epoch 1/100
898/898 [==============================] - 26s 27ms/step - loss: 2.2405 - accuracy: 0.2681 - val_loss: 1.5957 - val_accuracy: 0.3664
Epoch 2/100
898/898 [==============================] - 24s 27ms/step - loss: 1.5201 - accuracy: 0.4133 - val_loss: 1.7197 - val_accuracy: 0.3877
Epoch 3/100
898/898 [==============================] - 24s 27ms/step - loss: 1.3152 - accuracy: 0.4957 - val_loss: 1.2763 - val_accuracy: 0.5118
Epoch 4/100
898/898 [==============================] - 24s 27ms/step - loss: 1.1459 - accuracy: 0.5675 - val_loss: 1.3215 - val_accuracy: 0.4811
Epoch 5/100
898/898 [==============================] - 24s 27ms/step - loss: 1.0002 - accuracy: 0.6179 - val_loss: 1.4356 - val_accuracy: 0.4802
Epoch 6/100
898/898 [==============================] - 24s 27ms/step - loss: 0.7977 - accuracy: 0.7008 - val_loss: 1.2124 - val_accuracy: 0.5659
Epoch 7/100
898/898 [==============================] - 24s 27ms/step - loss: 0.6030 - accuracy: 0.7767 - val_loss: 1.3307 - val_ac

In [24]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.00001, 
                                           beta_1=0.97, beta_2=0.97, 
                                           epsilon=1e-07),
    metrics=['accuracy']
)
history = model.fit(X_train,y_train,
                    epochs=100,
                    shuffle=True,
                    validation_data=(X_test,y_test),
                    callbacks=callbacks)

Epoch 1/100
898/898 [==============================] - 26s 28ms/step - loss: 0.0793 - accuracy: 0.9739 - val_loss: 1.9703 - val_accuracy: 0.6021
Epoch 2/100
898/898 [==============================] - 24s 27ms/step - loss: 0.0588 - accuracy: 0.9806 - val_loss: 1.9854 - val_accuracy: 0.6046
Epoch 3/100
898/898 [==============================] - 24s 27ms/step - loss: 0.0544 - accuracy: 0.9841 - val_loss: 2.0174 - val_accuracy: 0.6043
Epoch 4/100
898/898 [==============================] - 24s 27ms/step - loss: 0.0487 - accuracy: 0.9846 - val_loss: 2.0419 - val_accuracy: 0.6073
Epoch 5/100
898/898 [==============================] - 24s 27ms/step - loss: 0.0426 - accuracy: 0.9856 - val_loss: 2.0744 - val_accuracy: 0.6071
Epoch 6/100
898/898 [==============================] - 24s 27ms/step - loss: 0.0387 - accuracy: 0.9875 - val_loss: 2.1125 - val_accuracy: 0.6080
Epoch 7/100
898/898 [==============================] - 25s 27ms/step - loss: 0.0342 - accuracy: 0.9894 - val_loss: 2.1403 - val_ac

In [ ]:
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

y_pred=model.predict_classes(X_test)
score=metrics.accuracy_score((np.argmax(y_test,axis=1)),y_pred)

loss_values = history.history['loss']
epochs = range(1, len(loss_values)+1)

plt.plot(epochs, loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

print("Nous avons une accuracy de : {}".format(score))

cm = metrics.confusion_matrix(np.argmax(y_test,axis=1), y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'viridis')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title("Matrice de confusion", size = 20)
plt.show()

In [26]:
model.save("/content/gdrive/My Drive/emotion_classification/emotion_classification_model")

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/emotion_classification/emotion_classification_model/assets
